In [ ]:
%matplotlib inline
import numpy as np
import os
from scipy import misc
import tensorflow as tf
import matplotlib.pyplot as plt

def generate_batch():
    filenames = tf.train.match_filenames_once("../data/LENS/images/*.jpeg")
    filename_queue = tf.train.string_input_producer(filenames)
    reader=tf.WholeFileReader()
    key, value=reader.read(filename_queue)
    img_uint8 = tf.image.decode_jpeg(value, channels=1)
    #img = tf.divide(img_uint8, np.float32(256))
    #img = img_uint8
    #print(img.dtype)
    
    #TODO:
    #When training cost is so much high maybe becuase of the way of conversion from uint8 to float32
    img = tf.cast(img_uint8, tf.float32)
    img.set_shape([256,256,1])
    # Subtract off the mean and divide by the variance of the pixels.
    #img = tf.image.per_image_standardization(img)
    # minimum number elements in the queue after a dequeue, used to ensure 
    # that the samples are sufficiently mixed I think 10 times the BATCH_SIZE is sufficient
    min_after_dequeue = 3 * BATCH_SIZE
    # the maximum number of elements in the queue
    capacity = 20 * BATCH_SIZE
    # shuffle the data to generate BATCH_SIZE samples
    data_batch = tf.train.shuffle_batch([img], batch_size=BATCH_SIZE, capacity=capacity, 
                                    min_after_dequeue=min_after_dequeue)
    return data_batch
    

data_batch = generate_batch()
#data_batch = np.reshape(batch, [10,256,256,1])

init = (tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord = coord)
    #print(sess.run(key))
    #for _ in range(1):
    imgs_batch = sess.run(data_batch)
    print(imgs_batch.shape)
    coord.request_stop()
    coord.join(threads)
    

    for img in imgs_batch:
        image = np.reshape(img, (256,256))
        
        f, ax = plt.subplots(1,2)
        ax[0].imshow(image, cmap='Greys_r')
        ax[1].imshow(image, cmap='Greys_r')